In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks


,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [3]:
from secret import IEX_CLOUD_API_TOKEN

In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 77277214, 'calculationPrice': 'tops', 'change': 1.644, 'changePercent': 0.01103, 'close': 0, 'closeSource': 'liiaffoc', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': 0, 'highSource': 'tri ema IElcrei pXe', 'highTime': 1688872128102, 'iexAskPrice': 155.26, 'iexAskSize': 152, 'iexBidPrice': 155.69, 'iexBidSize': 414, 'iexClose': 150.945, 'iexCloseTime': 1671173172496, 'iexLastUpdated': 1701037653646, 'iexMarketPercent': 0.010769153870126105, 'iexOpen': 153.154, 'iexOpenTime': 1662570943375, 'iexRealtimePrice': 150.837, 'iexRealtimeSize': 6, 'iexVolume': 82660, 'lastTradeTime': 1673385298686, 'latestPrice': 157.066, 'latestSource': 'IEX real time price', 'latestTime': '9:43:28 AM', 'latestUpdate': 1639276414395, 'latestVolume': None, 'low': 0, 'lowSource': 'r pEeeec i rlmtIaXi', 'lowTime': 16

In [5]:
price = data['latestPrice']
market_cap = data['marketCap']
print(market_cap)

2591488528202


In [6]:
my_columns = ['Ticker','stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,stock Price,Market Capitalization,Number of Shares to Buy


In [7]:
final_dataframe.append(
    pd.Series([
       symbol,
        price,
        market_cap,
        'N/A'
        
    ],
     index=my_columns,
    ),
   
    ignore_index = True
)

,Ticker,stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,157.066,2591488528202,N/A


In [9]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe=final_dataframe.append(
        pd.Series([
            stock,
            data['latestPrice'],
            data['marketCap'],
            'N/A'
        ],
        index=my_columns
        ),
        ignore_index = True
    )
final_dataframe

,Ticker,stock Price,Market Capitalization,Number of Shares to Buy
0,A,165.100,49675554600,N/A
1,AAL,20.349,12789188887,N/A
2,AAP,235.740,15119312640,N/A
3,AAPL,150.010,2581150311670,N/A
4,ABBV,108.560,200761120853,N/A
...,...,...,...,...
500,YUM,126.550,38774595752,N/A
501,ZBH,153.420,31688131293,N/A
502,ZBRA,541.950,29375018252,N/A
503,ZION,68.670,10773187889,N/A


In [10]:
def chunks(lst, n):
    """ Yield succesive n-sized chunk from lst"""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [11]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    #print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns= my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string},fb,tsla&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['marketCap'],
                'N/A'
            ],
            index = my_columns),
            ignore_index = True
            )
final_dataframe       

,Ticker,stock Price,Market Capitalization,Number of Shares to Buy
0,A,163.150,48579822580,N/A
1,AAL,20.400,12953279373,N/A
2,AAP,240.870,14974643544,N/A
3,AAPL,155.700,2530214198737,N/A
4,ABBV,112.780,194564868596,N/A
...,...,...,...,...
500,YUM,127.025,38866991489,N/A
501,ZBH,154.400,32682917779,N/A
502,ZBRA,558.326,28479651607,N/A
503,ZION,66.650,11239958338,N/A


In [12]:
portfolio_size = input('Enter the value of your portfolio: ')
try:
    val = float(portfolio_size)
except ValueError:
    print('this is not a number! \n please try again')
    portfolio_size = input('Enter the value of your portfolio: ')
    val = float(portfolio_size)

Enter the value of your portfolio: 10000000


In [13]:
position_size = val/len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i,'Number of Shares to Buy']= math.floor(position_size/final_dataframe.loc[i, 'stock Price'])
    
final_dataframe

,Ticker,stock Price,Market Capitalization,Number of Shares to Buy
0,A,163.150,48579822580,121
1,AAL,20.400,12953279373,970
2,AAP,240.870,14974643544,82
3,AAPL,155.700,2530214198737,127
4,ABBV,112.780,194564868596,175
...,...,...,...,...
500,YUM,127.025,38866991489,155
501,ZBH,154.400,32682917779,128
502,ZBRA,558.326,28479651607,35
503,ZION,66.650,11239958338,297


In [21]:
writer = pd.ExcelWriter('my trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, 'my trades', index=False)

In [22]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
    'font_color': '#ffffff',
    'bg_color': background_color,
    'border':1
}
)
dollar_format = writer.book.add_format({
    'num_format': '$0.00',
    'font_color': '#ffffff',
    'bg_color': background_color,
    'border': 1
}
)
integer_format = writer.book.add_format({
    'num_format': '0',
    'font_color': '#ffffff',
    'bg_color': background_color,
    'border': 1
}
)

In [23]:
#writer.sheets['my trades'].set_column('A:A', 18, string_format)
#writer.sheets['my trades'].set_column('B:B', 18, string_format)
#writer.sheets['my trades'].set_column('C:C', 18, string_format)
#writer.sheets['my trades'].set_column('D:D', 18, string_format)
#writer.save()
#writer.sheets['my trades'].write('A1', 'Ticker',string_format)
#writer.sheets['my trades'].write('B1','Stock Price',string_format)
#writer.sheets['my trades'].write('C1', 'market capitalization',string_format)
#writer.sheets['my trades'].write('D1', 'Number of Shares to Buy ',string_format)

In [25]:
column_formats = {
    "A":['Ticker', string_format],
    "B":['stock Price', dollar_format],
    "C":['market Capitalization', dollar_format],
    "D":['Number of Shares to Buy', integer_format]
}
for column in column_formats.keys():
    writer.sheets['my trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['my trades'].write(f'{column}1', column_formats[column][0] ,column_formats[column][1])
    
writer.save()